In [1]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o-mini")

In [2]:
# Define la plantilla del prompt
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un experto en reseñas de productos."),
        ("human", "Enumera las principales características del producto {product_name}."),
    ]
)

In [3]:
# Define el paso de análisis de pros
def analizar_pros(caracteristicas):
    plantilla_pros = ChatPromptTemplate.from_messages(
        [
            ("system", "Eres un experto en reseñas de productos."),
            (
                "human",
                "Dadas estas características: {caracteristicas}, enumera los pros de estas características.",
            ),
        ]
    )
    return plantilla_pros.format_prompt(caracteristicas=caracteristicas)


# Define el paso de análisis de contras
def analizar_contras(caracteristicas):
    plantilla_contras = ChatPromptTemplate.from_messages(
        [
            ("system", "Eres un experto en reseñas de productos."),
            (
                "human",
                "Dadas estas características: {caracteristicas}, enumera los contras de estas características.",
            ),
        ]
    )
    return plantilla_contras.format_prompt(caracteristicas=caracteristicas)


In [4]:
# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

In [5]:

# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analizar_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analizar_contras(x)) | model | StrOutputParser()
)

In [6]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

In [7]:
# Run the chain
result = chain.invoke({"product_name": "MacBook Pro"})

In [8]:
# Output
print(result)

Pros:
Claro, aquí tienes una lista de los pros de las características del MacBook Pro:

1. **Pantalla**: 
   - **Calidad visual excepcional**: La pantalla Retina con tecnología True Tone ofrece colores vibrantes y una claridad impresionante, ideal para diseño gráfico y edición de video.
   - **Brillo y resolución**: Con hasta 500 nits de brillo y alta resolución, proporciona una experiencia visual óptima en diversas condiciones de iluminación.

2. **Procesador**:
   - **Rendimiento superior**: Los procesadores Apple Silicon proporcionan un rendimiento excepcional en tareas intensivas, como edición de video y desarrollo de software.
   - **Eficiencia energética**: Los chipsets Apple son más eficientes, lo que contribuye a una mayor duración de la batería.

3. **Memoria RAM**:
   - **Capacidad expandible**: La opción de hasta 64 GB de RAM permite manejar múltiples aplicaciones y procesos exigentes simultáneamente sin comprometer el rendimiento.

4. **Almacenamiento**:
   - **Velocidad y 